Laboratorio 4(Regresion logistica One vs All) Grupo 1
Implementacion del modelo de regresion logistica one-vs-all para realizar clasificacion de clases.


NOMBRE: LUIS FERNANDO AMADOR YUCRA 

CARRERA: INGENIERIA DE SISTEMAS

FECHA: 09/09/2024

## 1 Clasificación multiclase




In [877]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

import pandas as pd

#esta tabulate nos sirve para hacer tablas
from tabulate import tabulate

# Librerias para graficación (trazado de gráficos)
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D  # -> Necesario para graficar superficies 3D

#Para separa el 20% y 80%
from sklearn.model_selection import train_test_split

# Modulo de optimizacion en scipy
from scipy import optimize

In [878]:
#from google.colab import drive
#drive.mount('/content/drive')



### 1.1 Cargamos los datos del dataset

 
cargamos los datos haciendo el uso de la libreria Pandas que es una herramienta poderosa y versátil utilizada para manipulación y análisis de datos.

In [879]:
data = pd.read_csv('credit_score.csv', delimiter=',')

data

,Unnamed: 0,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,1,2,23.000000,13,19114.12,4194.170850,3.0,4.0,3.000000,4.0,...,1,809.98,31.944960,0,1,49.574949,118.280222,3,284.629162,2
1,2,6,34.429817,13,19114.12,4194.170850,3.0,4.0,3.000000,4.0,...,1,809.98,28.609352,267,1,49.574949,81.699521,4,331.209863,2
2,3,0,23.000000,13,19114.12,4194.170850,3.0,4.0,3.000000,4.0,...,1,809.98,31.377862,268,1,49.574949,199.458074,5,223.451310,2
3,4,7,23.000000,13,19114.12,1824.843333,3.0,4.0,3.000000,4.0,...,1,809.98,24.797347,269,1,49.574949,41.420153,1,341.489231,2
4,5,5,23.000000,13,19114.12,4194.170850,3.0,4.0,3.000000,4.0,...,1,809.98,27.262259,270,1,49.574949,62.430172,6,340.479212,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79800,99991,1,29.000000,1,20002.88,1929.906667,10.0,8.0,29.000000,5.0,...,0,3571.70,37.140784,75,2,60.964772,34.662906,0,337.362988,1
79801,99993,2,25.000000,9,39628.99,3359.415833,4.0,6.0,7.000000,2.0,...,1,502.38,29.135447,376,1,58638.000000,180.733095,4,400.104466,1
79802,99997,5,25.000000,9,39628.99,3359.415833,4.0,6.0,14.546679,2.0,...,1,502.38,41.255522,380,1,35.104023,24.028477,0,516.809083,0
79803,99998,4,25.000000,9,39628.99,3359.415833,4.0,6.0,7.000000,2.0,...,1,502.38,33.638208,381,1,35.104023,251.672582,3,319.164979,1


mostrando la tabla nos damos cuenta que nuestra Y a predicir es el Credit_Score, para tener una mejor vision de la cantidad de clases que existe, se hizo el siguiente codigo:

donde separamos nuestra columna Y con value_counts() este método de pandas cuenta el número de veces que aparece cada valor único en la columna Credit_Score del DataFrame. Devuelve una Serie pandas donde los índices son los valores únicos de la columna Credit_Score y los valores son el recuento de ocurrencias de cada valor.

In [880]:
# Eliminar la primera columna (índice 0)
data = data.drop(data.columns[0], axis=1)

data

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,2,23.000000,13,19114.12,4194.170850,3.0,4.0,3.000000,4.0,128,...,1,809.98,31.944960,0,1,49.574949,118.280222,3,284.629162,2
1,6,34.429817,13,19114.12,4194.170850,3.0,4.0,3.000000,4.0,128,...,1,809.98,28.609352,267,1,49.574949,81.699521,4,331.209863,2
2,0,23.000000,13,19114.12,4194.170850,3.0,4.0,3.000000,4.0,128,...,1,809.98,31.377862,268,1,49.574949,199.458074,5,223.451310,2
3,7,23.000000,13,19114.12,1824.843333,3.0,4.0,3.000000,4.0,128,...,1,809.98,24.797347,269,1,49.574949,41.420153,1,341.489231,2
4,5,23.000000,13,19114.12,4194.170850,3.0,4.0,3.000000,4.0,128,...,1,809.98,27.262259,270,1,49.574949,62.430172,6,340.479212,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79800,1,29.000000,1,20002.88,1929.906667,10.0,8.0,29.000000,5.0,4913,...,0,3571.70,37.140784,75,2,60.964772,34.662906,0,337.362988,1
79801,2,25.000000,9,39628.99,3359.415833,4.0,6.0,7.000000,2.0,683,...,1,502.38,29.135447,376,1,58638.000000,180.733095,4,400.104466,1
79802,5,25.000000,9,39628.99,3359.415833,4.0,6.0,14.546679,2.0,683,...,1,502.38,41.255522,380,1,35.104023,24.028477,0,516.809083,0
79803,4,25.000000,9,39628.99,3359.415833,4.0,6.0,7.000000,2.0,683,...,1,502.38,33.638208,381,1,35.104023,251.672582,3,319.164979,1


In [881]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79805 entries, 0 to 79804
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Month                     79805 non-null  int64  
 1   Age                       79805 non-null  float64
 2   Occupation                79805 non-null  int64  
 3   Annual_Income             79805 non-null  float64
 4   Monthly_Inhand_Salary     79805 non-null  float64
 5   Num_Bank_Accounts         79805 non-null  float64
 6   Num_Credit_Card           79805 non-null  float64
 7   Interest_Rate             79805 non-null  float64
 8   Num_of_Loan               79805 non-null  float64
 9   Type_of_Loan              79805 non-null  int64  
 10  Delay_from_due_date       79805 non-null  int64  
 11  Num_of_Delayed_Payment    79805 non-null  float64
 12  Changed_Credit_Limit      79805 non-null  int64  
 13  Num_Credit_Inquiries      79805 non-null  float64
 14  Credit

In [882]:
#contar cuantas clases tiene nuestra columna y
Class_counts= data["Credit_Score"].value_counts()

#Contar cuantas caracteristicas tiene nuestro dataset sin contar con la columna y 
feactures_counts= data.drop(["Credit_Score"],axis=1)
feactures_counts=feactures_counts.shape[1]

#Mostramos los datos con print

print(f"La cantidad de caracteristicas de nuestro dataset es: {feactures_counts}")
print(f'La cantidad de clases es: 3')
Class_counts

La cantidad de caracteristicas de nuestro dataset es: 23
La cantidad de clases es: 3


Credit_Score
1    42470
0    23129
2    14206
Name: count, dtype: int64

### 1.1.1 Analisis del dataset:
Hay 3 clases diferentes de puntajes crediticios: etiquetados como 0, 1 y 2.

La clase 1 parece ser la más común, seguida por la clase 0 y luego la clase 2.

Este análisis de la distribución de los puntajes crediticios en el conjunto de datos es útil para comprender la proporción de cada clase. Esto puede ser relevante al desarrollar un modelo de clasificación como la Regresión Logística one vs all.

In [883]:
#almacenamos la cantidad de caracteristicas en una variable
input_layer_size  = feactures_counts

#almacenamos la cantidad de clases en una variable

#para este caso no es necesario cambiar los valores de la ultima clase, ya que este cuenta con 0,1,2, 
# en caso de contener 1,2,3 se tendria que hacer los cambios necesarios para que sea 1,2,0, o simplememte suma 1 a la cantidad de clases.
num_labels = len(np.unique(data['Credit_Score']))
print(f"numero de clases: {num_labels}")

numero de clases: 3


### 1.1.2 Separacion de datos
Se debe tomar en cuenta que cada clase tiene su propia cantidad, por lo cual separarlos directamente en un 80% para entrenamiento y un 20% para test no seria tan efectivo, ya que puede que en el 80% hay mas datos de una clase que las otras, provocando que nuestro modelo no conozca mucho sobre esa clase, por lo cual debe separarse un 80% para entrenamiento y un 20% para pruebas de cada clase.

Para este hacemos uso de la libreria train_test_split

In [884]:
# hacemos uso del DataFrame llamado 'df' que contiene nuestros datos datos
# y es la columna que contiene las etiquetas de clase (en este caso, 'Credit_Score')

#creamos una variable temporal que contentra toda la columna de 'Credit_Score'
y_temp = data['Credit_Score']

# Para la clase 0

#donde y_temp es igual a 0, separamos los datos en train_class_0, test_class_0
data_class_0 = data[y_temp == 0]
train_class_0, test_class_0 = train_test_split(data_class_0, test_size=0.2, random_state=42)

# Para la clase 1

#donde y_temp es igual a 1, separamos los datos en train_class_1, test_class_1
data_class_1 = data[y_temp == 1]
train_class_1, test_class_1 = train_test_split(data_class_1, test_size=0.2, random_state=42)

# Para la clase 2

#donde y_temp es igual a 2, separamos los datos en train_class_2, test_class_2
data_class_2 = data[y_temp == 2]
train_class_2, test_class_2 = train_test_split(data_class_2, test_size=0.2, random_state=42)

haciendo conteo de separacion de datos:

In [885]:
print(f"Para la clase 0 tenemos una cantidad de: { data_class_0.shape[0]} donde el 80% es: {train_class_0.shape[0]} y el 20% es: {test_class_0.shape[0]}")
print(f"Para la clase 1 tenemos una cantidad de: { data_class_1.shape[0]} donde el 80% es: {train_class_1.shape[0]} y el 20% es: {test_class_1.shape[0]}")
print(f"Para la clase 2 tenemos una cantidad de: { data_class_2.shape[0]} donde el 80% es: {train_class_2.shape[0]} y el 20% es: {test_class_2.shape[0]}")
print(f"La cantidad total de datos es: {data.shape[0]}")

Para la clase 0 tenemos una cantidad de: 23129 donde el 80% es: 18503 y el 20% es: 4626
Para la clase 1 tenemos una cantidad de: 42470 donde el 80% es: 33976 y el 20% es: 8494
Para la clase 2 tenemos una cantidad de: 14206 donde el 80% es: 11364 y el 20% es: 2842
La cantidad total de datos es: 79805


Separamos cada uno en sus X_train, y_train, X_test y y_test respectivos:

In [886]:
#para la parte de entrenamiento, separamos las caracteristicas de la etiqueta

#ENTRENAMIENTO
#para la clase 0
X_train_class_0 = train_class_0.drop(['Credit_Score'], axis=1)
y_train_class_0 = train_class_0['Credit_Score']

#para la clase 1
X_train_class_1 = train_class_1.drop(['Credit_Score'], axis=1)
y_train_class_1 = train_class_1['Credit_Score']

#para la clase 2
X_train_class_2 = train_class_2.drop(['Credit_Score'], axis=1)
y_train_class_2 = train_class_2['Credit_Score']


#PRUEBAS
#ahora para la parte de pruebas, separamos las caracteristicas de la etiqueta
#para la clase 0
X_test_class_0 = test_class_0.drop(['Credit_Score'], axis=1)
y_test_class_0 = test_class_0['Credit_Score']

#para la clase 1
X_test_class_1 = test_class_1.drop(['Credit_Score'], axis=1)
y_test_class_1 = test_class_1['Credit_Score']

#para la clase 2
X_test_class_2 = test_class_2.drop(['Credit_Score'], axis=1)
y_test_class_2 = test_class_2['Credit_Score']

ahora unimos todos en una sola matriz para X y y: pero luego debemos mezclar los datos, haciendo uso de np.random.permutation(len(X)) genera un arreglo de índices permutados aleatoriamente. 
Luego, estos índices se usan para reorganizar tanto las características como las etiquetas de entrenamiento y prueba.

In [887]:
#separando los datos de entrenamiento y pruebas

#para los datos de entrenamiento
X_train = pd.concat([X_train_class_0, X_train_class_1, X_train_class_2]).values
y_train = pd.concat([y_train_class_0, y_train_class_1, y_train_class_2]).values


indices_train = np.random.permutation(len(X_train))
X_train = X_train[indices_train]
y_train = y_train[indices_train]
m_train = len(y_train)

#para los datos de pruebas
X_test = pd.concat([X_test_class_0, X_test_class_1, X_test_class_2]).values
y_test = pd.concat([y_test_class_0, y_test_class_1, y_test_class_2]).values

indices_test = np.random.permutation(len(X_test))
X_test = X_test[indices_test]
y_test = y_test[indices_test]
m_test = len(y_test)


In [888]:
#Visualizamos una cantidad de datos de entrenamiento
print('{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}'.format(
    'X[:,0]', 'X[:, 1]', 'X[:, 2]', 'X[:, 3]', 'X[:, 4]', 'X[:, 5]', 'X[:, 6]', 'X[:, 7]', 'X[:, 8]', 'X[:, 9]', 'X[:, 10]', 'X[:, 11]', 'X[:, 12]', 
    'X[:, 13]', 'X[:, 14]', 'X[:, 15]', 'X[:, 16]', 'X[:, 17]', 'X[:, 18]', 'X[:, 19]', 'X[:, 20]', 'X[:, 21]', 'X[:, 22]', 'Y'
))
print('-' * 250)

for i in range(10):
    print('{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:8.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:8.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}'.format(
        X_train[i, 0],
        X_train[i, 1],
        X_train[i, 2],
        X_train[i, 3],
        X_train[i, 4],
        X_train[i, 5],
        X_train[i, 6],
        X_train[i, 7],
        X_train[i, 8],
        X_train[i, 9],
        X_train[i, 10],
        X_train[i, 11],
        X_train[i, 12],
        X_train[i, 13],
        X_train[i, 14],
        X_train[i, 15],
        X_train[i, 16],
        X_train[i, 17],
        X_train[i, 18],
        X_train[i, 19],
        X_train[i, 20],
        X_train[i, 21],
        X_train[i, 22],
        y_train[i]
    ))

print(" ")
print('El 80% de ejemplos para entrenamiento son la cantidad de: {:.0f} de ejemplos'.format( len(X_train)))
print('El 20% de ejemplos para pruebas son la cantidad de: {:.0f} de ejemplos'.format( len(X_test)))
print('La cantidad total de ejemplos es de: {:.0f} de ejemplos'.format( len(data)))

    X[:,0]   X[:, 1]   X[:, 2]   X[:, 3]   X[:, 4]   X[:, 5]   X[:, 6]   X[:, 7]   X[:, 8]   X[:, 9]  X[:, 10]  X[:, 11]  X[:, 12]  X[:, 13]  X[:, 14]  X[:, 15]  X[:, 16]  X[:, 17]  X[:, 18]  X[:, 19]  X[:, 20]  X[:, 21]  X[:, 22]         Y
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
         2        27        14     82330      6732         9      10        20         3      2088        41        23      2315         8         0      1680        28     100         2       156        75         0       682         0
         0        28         7      7534      4194         8       7        25         5      4406        18        16      3230        11         0      2391        34       0         2        17        38         5       324         0
         7        39         4    

### 1.2 Normalización de caracteristicas
Al visualizar los datos se puede observar que las caracteristicas tienen diferentes magnitudes, por lo cual se debe transformar cada valor en una escala de valores similares, esto con el fin de que el descenso por el gradiente pueda converger mas rapidamente. Se aplica la normalizacion esto debido a que los datos de las X estan a diferentes escalas.

Hacemos el uso de la siguiente funcion para normalizar los datos de las columnas X:

In [889]:

def  featureNormalize(X):
    X_norm = X.copy()

    #creamos un array de ceros con una longitud igual al número de columnas en el array X. La variable mu y sigma se inicializa como este array de ceros.
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    #Creamos el promedio de cada fila de X
    #media de cada columna
    mu = np.mean(X, axis = 0)

    #desviacion estandar de cada fila de X
    sigma = np.std(X, axis = 0)

    sigma[sigma == 0] = 1

    #normalizamos los datos con la siguiente formula
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma


Normalizacion de Caracteristicas

In [890]:
X_norm, mu, sigma= featureNormalize(X_train)

#Visualizamos una cantidad de datos de entrenamiento
print('{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}'.format(
    'X[:,0]', 'X[:, 1]', 'X[:, 2]', 'X[:, 3]', 'X[:, 4]', 'X[:, 5]', 'X[:, 6]', 'X[:, 7]', 'X[:, 8]', 'X[:, 9]', 'X[:, 10]', 
    'X[:, 11]', 'X[:, 12]', 'X[:, 13]', 'X[:, 14]', 'X[:, 15]', 'X[:, 16]', 'X[:, 17]', 'X[:, 18]', 'X[:, 19]', 'X[:, 20]', 'X[:, 21]', 'X[:, 22]','Y'
))
print('-' * 250)

for i in range(10):
    print('{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:8.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:8.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}'.format(
        X_norm[i, 0],
        X_norm[i, 1],
        X_norm[i, 2], 
        X_norm[i, 3], 
        X_norm[i, 4], 
        X_norm[i, 5], 
        X_norm[i, 6], 
        X_norm[i, 7], 
        X_norm[i, 8], 
        X_norm[i, 9], 
        X_norm[i, 10],
        X_norm[i, 11],
        X_norm[i, 12], 
        X_norm[i, 13], 
        X_norm[i, 14], 
        X_norm[i, 15], 
        X_norm[i, 16],
        X_norm[i, 17],
        X_norm[i, 18],
        X_norm[i, 19], 
        X_norm[i, 20], 
        X_norm[i, 21], 
        X_norm[i, 22], 
        y_train[i]
    ))

    X[:,0]   X[:, 1]   X[:, 2]   X[:, 3]   X[:, 4]   X[:, 5]   X[:, 6]   X[:, 7]   X[:, 8]   X[:, 9]  X[:, 10]  X[:, 11]  X[:, 12]  X[:, 13]  X[:, 14]  X[:, 15]  X[:, 16]  X[:, 17]  X[:, 18]  X[:, 19]  X[:, 20]  X[:, 21]  X[:, 22]         Y
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        -1        -1         1        -0         1         1       1         1        -0        -1         1        -0        -0        -0        -2         0        -1      -1         1        -0        -0        -2         1         0
        -2        -1        -0        -0        -0         1       0         1         1         0        -0        -0         1        -0        -2         1         0      -2         1        -0        -0         1        -0         0
         2         0        -1    

### 1.3 Creacion de la funcion Sigmoide
También conocida como la función logística, es una función matemática que toma cualquier número real como entrada y devuelve un valor en el rango de 0 a 1. Donde nuestra Z es nuestra hipotesis.

Creando la funcion:

In [891]:
def sigmoid(z):
    z = np.array(z)

    g = np.zeros(z.shape)

    g = 1 / (1 + np.exp(-z))

    return g

### 1.4 Funcion de costo
Aplicando la teoria donde nuestra funcion recibira parametros como Theta, x, y y lamda_, donde lamda_ es nuestro parametro de regularizacion.

Donde la funcion nos devuelve un costo y nuestro gradiente.

In [892]:
def lrCostFunction(theta, X, y, lambda_):
    #creamos una variable m que contiene la longitud de y
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    #inicializamos J y grad
    J = 0
    grad = np.zeros(theta.shape)

    #calculamos h haciendo uso de la funcion sigmoid, donde h es nuestra hipotesis
    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
    
    grad = grad + (lambda_ / m) * temp

    return J, grad

### 1.5 Clasificacion One-vs-All
Aqui estariamos usando un ciclo for para iterar sobre cada una de las clases, luego haciendo uso de optimize.minimize que es un método de la biblioteca scipy que encuentra el mínimo de una función. En este caso, se trata de minimizar la función de costos de regresión logística (lrCostFunction).

Los parámetros iniciales (initial_theta). Una tupla que contiene los datos de entrenamiento (X), las etiquetas (y == c), y el parámetro de regularización (lambda_). jac=True indica que la función de coste devuelve tanto el coste como el gradiente. El método de optimización ('CG' significa Gradiente Conjugado). El diccionario de opciones (options) que se establece en 1000 para limitar el número máximo de iteraciones del optimizador.

In [893]:
def oneVsAll(X, y, num_labels, lambda_):
    # algunas variables utiles
    #m es la longitud de y, n es la cantidad de columnas en X
    m, n = X.shape

    # inicializamos la matriz de thetas con ceros, y con n+1 columnas que seria nuestro sesgo
    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    # iteramos sobre cada etiqueta (clase) y entrenamos un clasificador
    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 1000}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

Inicializamos nuestra lambda con valor de 0.1, usamos la funcion de oneVsAll donde pasamos los parametros de X_norm, num_labels que seria la cantidad de clases que tenemos, y nuestro lanbda para asi obtener nuestros Thetas para cada clase.

In [894]:
lambda_ = 0.1
all_theta = oneVsAll(X_norm, y_train, num_labels, lambda_)
print(all_theta.shape)

(3, 24)


In [895]:
for i in range(all_theta.shape[1]):
    print("Thetha",i," para la clase 0:", all_theta[0,i]," ; "," para la clase 1:", all_theta[1,i]," ; ", " para la clase 2:", all_theta[2,i])

Thetha 0  para la clase 0: -1.0815419166293518  ;   para la clase 1: 0.14285321980775767  ;   para la clase 2: -2.371067513704147
Thetha 1  para la clase 0: 0.0014111007161773197  ;   para la clase 1: 0.01685992777106229  ;   para la clase 2: -0.037373182620319904
Thetha 2  para la clase 0: -0.060719032888794974  ;   para la clase 1: 0.007170053780312108  ;   para la clase 2: 0.04983295197086631
Thetha 3  para la clase 0: 0.007170434687797648  ;   para la clase 1: -0.0011672223262592886  ;   para la clase 2: -0.014876744222946513
Thetha 4  para la clase 0: -0.024927485648445838  ;   para la clase 1: 0.02115265691766926  ;   para la clase 2: -0.0010965902576486296
Thetha 5  para la clase 0: -0.09457428802468652  ;   para la clase 1: -0.00429380036083792  ;   para la clase 2: 0.12333192689839385
Thetha 6  para la clase 0: -0.02054364945323232  ;   para la clase 1: 0.15972703064188337  ;   para la clase 2: -0.04488489749334661
Thetha 7  para la clase 0: 0.15747397513332873  ;   para la cl

### 1.5.1 Prediccion One-vs-all
Aqui creamos solo la funcion, donde mandamos los parametros de all_theta y la X que en este caso puede ser las X de prueba, pero antes deben de estar normalizadas.

In [896]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # añadimos unos a la matriz de X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

### 1.6 Validaciones

Para este caso de validacion se hizo uso de la funcion de predictOneVsAll() creada anteriormente, los siguiente fue normalizar nuestra X_test que son el 20% para pruebas, ahora se procedio a normalizarlo haciendo uso de mu y sigma calculado anteriormente en la funcion de normalizacion.

luego se hace uso de np.mean donde nos calcula el promedio de los valores booleanos en el array resultante de la comparación.

### 1.6.1 Con los datos de entrenamiento
Para este caso se uso los datos de X_norm usando la funcion de predictOneVsAll(), viendo el resultado de precision nos da un 62% el cual esta muy lejos de ser un buen modelo.

In [897]:
# print(X_test.shape)
pred_test= predictOneVsAll(all_theta, X_norm)
print('Precision del conjunto de entrenamiento: {:.2f}%'.format(np.mean(pred_test== y_train) * 100))

Precision del conjunto de entrenamiento: 62.00%


### 1.6.2 con los datos de prueba
Para este caso se uso los datos de X_test, luego se procedio a normalizar los datos.

Usando la funcion de predictOneVsAll(), viendo el resultado de precision nos da un 62.24% , un poco en diferencia a la anterior prediccion.

In [898]:
X_test1 = X_test.copy()
# print(X_test.shape)
X_test1 = (X_test1 - mu) / sigma
pred_train = predictOneVsAll(all_theta, X_test1)
print('Precision del conjunto de prueba: {:.2f}%'.format(np.mean(pred_train == y_test) * 100))

Precision del conjunto de prueba: 62.24%
